In [5]:
import gcsfs
import os
import allel
import zarr
import pandas as pd
import numpy

In [6]:
gcs_orig = gcsfs.GCSFileSystem(project='malariagen-jupyterhub', token='cache')
gcs =  gcsfs.GCSFileSystem(project='malariagen-jupyterhub', token=gcs_orig.session.credentials)

In [7]:
phase2path = os.path.join("ag1000g-release", "phase2.AR1", "variation", "main", "zarr", "biallelic", "ag1000g.phase2.ar1.pass.biallelic")

In [8]:
gcsmap = gcsfs.mapping.GCSMap(phase2path, gcs=gcs)

In [9]:
calldata= zarr.Group(gcsmap, read_only=True)

In [10]:
g = calldata["3L"]["calldata/GT"]

In [11]:
metadata = pd.read_csv("samples.meta.txt", sep="\t")

In [12]:
pop_select = metadata.population.isin({'BFgam', 'BFcol', 'GHcol', 'AOcol'}).values
pop_select

array([ True,  True,  True, ..., False, False, False])

In [23]:
pop_subset = metadata[pop_select]
pop_subset.head()

,ox_code,src_code,population,country,location,site,contributor,contact,year,m_s,sex,n_sequences,mean_coverage,ebi_sample_acc,latitude,longitude
0,AA0040-C,Twifo_Praso__E2,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,M,F,95033368,30.99,ERS311878,5.60858,-1.54926
1,AA0041-C,Twifo_Praso__H3,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,M,F,95843804,31.70,ERS311886,5.60858,-1.54926
2,AA0042-C,Takoradi_C7,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,107420666,35.65,ERS311894,4.91217,-1.77397
3,AA0043-C,Takoradi_H8,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,95993752,29.46,ERS311902,4.91217,-1.77397
4,AA0044-C,Takoradi_D10,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,103044262,33.67,ERS311910,4.91217,-1.77397


In [13]:
geno = allel.GenotypeChunkedArray(g)
geno

<GenotypeChunkedArray shape=(7897666, 1142, 2) dtype=int8 chunks=(524288, 61, 2)
   nbytes=16.8G cbytes=-1 cratio=-18038269144.0
   compression=blosc compression_opts={'cname': 'zstd', 'clevel': 1, 'shuffle': -1, 'blocksize': 0}
   values=zarr.core.Array>

In [15]:
pos_all = allel.SortedIndex(calldata["3L"]["variants/POS"])
pos_all

0,1,2,3,4,...,7897661,7897662,7897663,7897664,7897665
9790,9791,9798,9812,9815,...,41956530,41956532,41956537,41956541,41956551


In [17]:
geno_subset = geno.subset(sel1=pop_select)
geno_subset

<GenotypeChunkedArray shape=(7897666, 300, 2) dtype=int8 chunks=(7713, 300, 2)
   nbytes=4.4G cbytes=189.0M cratio=23.9
   compression=blosc compression_opts={'cname': 'lz4', 'clevel': 5, 'shuffle': 1, 'blocksize': 0}
   values=zarr.core.Array>

In [33]:
grp = pop_subset.groupby("population")

In [37]:
grp_indices = grp.indices
grp_indices

{'AOcol': array([222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234,
        235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247,
        248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260,
        261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273,
        274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286,
        287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299]),
 'BFcol': array([ 56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,
         72,  73,  74,  75,  76,  77,  78,  82,  83,  84,  94,  95,  96,
         97,  98, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 150, 155, 156, 158, 159, 160, 163, 164, 165, 166,
        167, 168, 169, 171, 175, 178, 181, 183, 184, 186, 187, 188, 189,
        190, 196, 197, 200, 201, 204, 205, 213, 216, 219]),
 'BFgam': array([ 55,  69,  70,  71,  79,  80,  81,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  99, 100, 10

In [39]:
ac_subpops = geno_subset.count_alleles_subpops(grp_indices)
ac_subpops

<AlleleCountsChunkedTable shape=(7897666,) dtype=[('AOcol', '<i4', (2,)), ('BFcol', '<i4', (2,)), ('BFgam', '<i4', (2,)), ('GHcol', '<i4', (2,))]
   nbytes=241.0M cbytes=24.3M cratio=9.9
   values=allel.chunked.storage_zarr.ZarrTable>

In [48]:
abba = allel.patterson_d(ac_subpops['AOcol'], ac_subpops['BFcol'], ac_subpops['BFgam'], ac_subpops['GHcol'])
abba

(array([ 0.,  0.,  0., ...,  0., -0.,  0.]),
 array([0., 0., 0., ..., 0., 0., 0.]))